In [1]:
!gunzip /content/SMSSpamCollection_txt_train.gz

gzip: /content/SMSSpamCollection_txt_train.gz: No such file or directory


In [2]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

In [3]:
import tensorflow as tf
import tensorflow_hub as hub

In [4]:
import pandas as pd
f = open('/content/SMSSpamCollection_txt_train')
df = pd.read_table(f, names=['target', 'message'])

In [5]:
# 1 если спам, 0 если не спам
df['target'] = df['target'].apply(lambda x: 1 if x=='spam' else 0)
target = df.pop('target')
# делим датасет на тренировочную и тестовую выборки
df_train = df.loc[:4000]
target_train = target.loc[:4000]
df_test = df.loc[4001:]
target_test = target.loc[4001:]

In [6]:
# датасеты с разными типами данных не могут быть сконкатенированны, поэтому оставлю как заготовку
import numpy

texts = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor(df))
texts = texts.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
texts = texts.batch(10)

labels = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor(target))
labels = labels.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
labels = labels.batch(10)

In [7]:
# посмотрим на входные данные
for text_batch in texts.take(1):
  for i in range(3):
    print(text_batch.numpy()[i])

for label_batch in labels.take(1):
  for i in range(3):
    print(label_batch.numpy()[i])

[b'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...']
[b'Ok lar... Joking wif u oni...']
[b"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
0
0
1


In [8]:
# используем предобученный слой эмбеддинга
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [9]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dropout(.2, input_shape=(10,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
history = model.fit(df,
                    target,
                    epochs=5,
                    verbose=1)

Epoch 1/5
157/157 [==============================] - 10s 58ms/step - loss: 0.3785 - accuracy: 0.8654
Epoch 2/5
157/157 [==============================] - 8s 54ms/step - loss: 0.1667 - accuracy: 0.8662
Epoch 3/5
157/157 [==============================] - 9s 58ms/step - loss: 0.0891 - accuracy: 0.9532
Epoch 4/5
157/157 [==============================] - 8s 53ms/step - loss: 0.0401 - accuracy: 0.9850
Epoch 5/5
157/157 [==============================] - 8s 49ms/step - loss: 0.0213 - accuracy: 0.9920


In [12]:
results = model.evaluate(df_test, target_test, verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

32/32 - 0s - loss: 0.0052 - accuracy: 1.0000 - 404ms/epoch - 13ms/step
loss: 0.005
accuracy: 1.000


In [13]:
model.save_weights('./checkpoints/my_checkpoint')